# Proyecto Final - Ensemble Learning (Aprendizaje por ensamblado)
## Parte 2 - Implementación y predicción

**Curso:** Statistical Learning

**Catedrático:** Ing. Luis Leal

**Estudiante:** Dany Rafael Díaz Lux (21000864)

**Objetivo:** Hacer clasificación binaria para determinar si una persona sobrevive o no del hundimiento del Titanic.

In [137]:
# Import required libraries
#from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#import datetime as dt
import joblib
#import matplotlib.pylab as plt
import numpy as np 
#import os.path
import pandas as pd
import scipy.stats as st
import sklearn.metrics as mts
#import tensorflow as tf
#print('Tensor flow version: ' + tf.__version__)

## Cargar información de validación

In [262]:
datosDeValidacion = pd.read_csv('datos_validacion.csv')
display(datosDeValidacion)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,771,"Lievens, Mr. Rene Aime",24.0,0,0,345781,9.5000,NaN,S,Lower,M,N
1,179,"Hale, Mr. Reginald",30.0,0,0,250653,13.0000,NaN,S,Middle,M,N
2,787,"Sjoblom, Miss. Anna Sofia",18.0,0,0,3101265,7.4958,NaN,S,Lower,F,Y
3,160,"Sage, Master. Thomas Henry",NaN,8,2,CA. 2343,69.5500,NaN,S,Lower,M,N
4,657,"Radeff, Mr. Alexander",NaN,0,0,349223,7.8958,NaN,S,Lower,M,N
...,...,...,...,...,...,...,...,...,...,...,...,...
174,81,"Waelens, Mr. Achille",22.0,0,0,345767,9.0000,NaN,S,Lower,M,N
175,456,"Jalsevac, Mr. Ivan",29.0,0,0,349240,7.8958,NaN,C,Lower,M,Y
176,137,"Newsom, Miss. Helen Monypeny",19.0,0,2,11752,26.2833,D47,S,Upper,F,Y
177,630,"O'Connell, Mr. Patrick D",NaN,0,0,334912,7.7333,NaN,Q,Lower,M,N


## Pre-procesar los datos de validación

### Ignorar columnas identificadoras

In [263]:
# Omitir columnas "PassengerId", "Name", "Ticket" y "Cabin" por ser columnas que tratan de distinguir a cada individuo y
# no buscan indicar una característica general.
caracteristicas = datosDeValidacion.iloc[:,[2,3,4,6,8,9,10]]
etiquetas = datosDeValidacion.iloc[:,11]
display(caracteristicas.head())
display(etiquetas.head())

,Age,SibSp,Parch,Fare,Embarked,passenger_class,passenger_sex
0,24.0,0,0,9.5000,S,Lower,M
1,30.0,0,0,13.0000,S,Middle,M
2,18.0,0,0,7.4958,S,Lower,F
3,NaN,8,2,69.5500,S,Lower,M
4,NaN,0,0,7.8958,S,Lower,M


0    N
1    N
2    Y
3    N
4    N
Name: passenger_survived, dtype: object

### Detectar columnas con información faltante (NaN), determinar porcentaje, e imputación.

In [264]:
# Listar columnas con valores NaN
columnasConNaN = caracteristicas.columns[caracteristicas.isna().any()].tolist()
for columna in columnasConNaN:
    print('Porcentaje NaN en ', columna, ':', round(100 * \
            len(caracteristicas[caracteristicas[columna].isna()]) / len(caracteristicas), 2), '%')

Porcentaje NaN en  Age : 20.11 %
Porcentaje NaN en  Embarked : 0.56 %


In [265]:
# Se imputará la información faltante con la media de los datos en la columna "Age"
caracteristicas['Age'].fillna(value=caracteristicas['Age'].mean(), inplace=True)

C:\Users\Dany\anaconda3\envs\py_galileo_2021\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [266]:
# Se imputará la información faltante con la moda de los datos en la columna "Embarked" por ser categórica
caracteristicas['Embarked'].fillna(value=caracteristicas['Embarked'].mode()[0], inplace=True)

### One hot encoding en columnas categóricas

In [267]:
# Realizar one hot encoding en columnas: "Enbarked", "passenger_class", "passenger_sex" y "passenger_survived"
caracteristicasConOhe = caracteristicas.join(pd.get_dummies(caracteristicas.Embarked, prefix='Embarked'))
caracteristicasConOhe = caracteristicasConOhe.join(pd.get_dummies(caracteristicasConOhe.passenger_class, prefix='passenger_class'))
caracteristicasConOhe = caracteristicasConOhe.join(\
                                pd.get_dummies(caracteristicasConOhe.passenger_sex, prefix='passenger_sex', drop_first=True))
caracteristicasConOhe = caracteristicasConOhe.loc[:, ~caracteristicasConOhe.columns.isin(['Embarked', 'passenger_class', 'passenger_sex'])]
etiquetasConOhe = pd.get_dummies(etiquetas, prefix='passenger_survived', drop_first=True)
display(caracteristicasConOhe.head())
display(etiquetasConOhe.head())

,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper,passenger_sex_M
0,24.000000,0,0,9.5000,0,0,1,1,0,0,1
1,30.000000,0,0,13.0000,0,0,1,0,1,0,1
2,18.000000,0,0,7.4958,0,0,1,1,0,0,0
3,30.453986,8,2,69.5500,0,0,1,1,0,0,1
4,30.453986,0,0,7.8958,0,0,1,1,0,0,1


,passenger_survived_Y
0,0
1,0
2,1
3,0
4,0


### Escalar características numéricas

In [268]:
# Se aplicará estandarización de datos como método de escalado
def estandarizar(x, mediaEntrenamiento = None, desviacionEntrenamiento = None):
    if(mediaEntrenamiento == None or desviacionEntrenamiento == None):
        # Si no se reciben datos de entrenamiento, se calculan de los features "x" recibidos
        media = np.mean(x)
        desviacion = np.std(x)
    else:
        media = mediaEntrenamiento
        desviacion = desviacionEntrenamiento
    
    return (x - media) / (desviacion), media, desviacion

### Cargar media y desviación de datos de entrenamiento para escalado de datos de validación

In [269]:
informacionEstadisticaEntrenamiento = dict(np.load('modelos/InformacionEstadistica.npy', allow_pickle=True)[0])

In [277]:
mediaAge_train = informacionEstadisticaEntrenamiento['mediaAge_train']
desviacionAge_train = informacionEstadisticaEntrenamiento['desviacionAge_train']
mediaSibSp_train = informacionEstadisticaEntrenamiento['mediaSibSp_train']
desviacionSibSp_train = informacionEstadisticaEntrenamiento['desviacionSibSp_train']
mediaParch_train = informacionEstadisticaEntrenamiento['mediaParch_train']
desviacionParch_train = informacionEstadisticaEntrenamiento['desviacionParch_train']
mediaFare_train = informacionEstadisticaEntrenamiento['mediaFare_train']
desviacionFare_train = informacionEstadisticaEntrenamiento['desviacionFare_train']

In [279]:
# Aplicar estandarización a columnas "Age", "SibSp", "Parch" y "Fare"
caracteristicasConOheEstandarizadas = caracteristicasConOhe.join(pd.DataFrame())
caracteristicasConOheEstandarizadas['Age'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas['Age'], mediaAge_train, desviacionAge_train)
caracteristicasConOheEstandarizadas['SibSp'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas['SibSp'], mediaSibSp_train, desviacionSibSp_train)
caracteristicasConOheEstandarizadas['Parch'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas['Parch'], mediaParch_train, desviacionParch_train)
caracteristicasConOheEstandarizadas['Fare'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas['Fare'], mediaFare_train, desviacionFare_train)
display(caracteristicasConOheEstandarizadas.head())

,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper,passenger_sex_M
0,-0.421627,-0.474164,-0.485553,-0.462055,0,0,1,1,0,0,1
1,0.054441,-0.474164,-0.485553,-0.392896,0,0,1,0,1,0,1
2,-0.897696,-0.474164,-0.485553,-0.501657,0,0,1,1,0,0,0
3,0.090462,6.443769,2.003451,0.724512,0,0,1,1,0,0,1
4,0.090462,-0.474164,-0.485553,-0.493753,0,0,1,1,0,0,1


## Cargar información de modelos

In [238]:
# Informacion de modelos
directorioModelos = 'modelos/'
extensionModelos = '.modelo'
nombreModeloArbolDecision = 'ArbolDecision_criterio=gini_profundidadMaxima=4'
nombreModeloSVM = 'SVM_regularizacionC=1.0_kernel=poly_grado=3'
nombreParametrosRegresionLogistica = 'RegresionLogisticaMiniBatchGradientDescent_epochs=500_lr=0.01_batchSize=32_alpha=0.0'
nombreProbabilidadesNaiveBayes = 'NaiveBayes'

In [239]:
# Cargar modelos
modeloArbolDecision = joblib.load(directorioModelos + nombreModeloArbolDecision + extensionModelos)
modeloSVM = joblib.load(directorioModelos + nombreModeloSVM + extensionModelos)
parametrosRegresionLogistica = np.load(directorioModelos + nombreParametrosRegresionLogistica + extensionModelos + '.npy')
probabilidadesNaiveBayes = dict(np.load(directorioModelos + nombreProbabilidadesNaiveBayes + extensionModelos + '.npy', allow_pickle=True)[0])

In [240]:
# Funciones auxiliares para modelo de regresión logística
def agregarColumnaUnos(x):
    if len(x.shape) < 2:
        x = np.array(x, dtype=np.float32).reshape(x.shape[0], -1)
    return np.append(x, np.ones(x.shape[0]).reshape(-1,1), 1)

def calcularSigmoid(x):
    return 1/(1 + np.exp(-x))

def predecirRegresionLogistica(X, parametros):
    X = agregarColumnaUnos(X)
    logits = np.matmul(X, parametros)
    logits = logits.reshape(logits.shape[0])
    return np.round(calcularSigmoid(logits))

# Función que indicará si una feature en X es categórica o no
def esCategorica(X, indexFeature):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
        
    if(len(np.unique(X[:,indexFeature])) < 11 and\
        len(np.unique(X[:,indexFeature]))/len(X[:,5]) < 0.4):
        return True
    return False

def predecirNaiveBayes(diccionarioProbabilidades, X):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
    etiquetas = diccionarioProbabilidades['__labels__']
    y_pred = np.zeros(shape=(X.shape[0],1), dtype=np.float32)
    for i in range(X.shape[0]):
        resultados = np.ones_like(etiquetas, dtype=np.float32)
        for indiceEtiqueta, etiqueta in enumerate(etiquetas):
            resultados[indiceEtiqueta] = diccionarioProbabilidades[str(etiqueta)]
            for indiceCaracteristica in range(X.shape[1]):
                if(esCategorica(X, indiceCaracteristica)):
                    if(str(indiceCaracteristica) + '_' + str(round(X[i,indiceCaracteristica], 10)) + '|' + str(etiqueta) \
                        in diccionarioProbabilidades.keys()):
                        resultados[indiceEtiqueta] *= \
                            diccionarioProbabilidades[str(indiceCaracteristica) + '_' + \
                                                      str(round(X[i,indiceCaracteristica], 10)) + '|' + str(etiqueta)]
                    else:
                        resultados[indiceEtiqueta] = 0.0
                        break
                # Para características continuas se obtiene la probabilidad del valor (en un rango pequeño) para valor dado
                else:
                    if('media_' + str(indiceCaracteristica) + '|' + str(etiqueta) \
                        in diccionarioProbabilidades.keys()):
                        margen = 0.001
                        valorZ = (X[i,indiceCaracteristica] \
                                  - diccionarioProbabilidades['media_' + str(indiceCaracteristica) + '|' + str(etiqueta)])\
                                    / diccionarioProbabilidades['desviacion_' + str(indiceCaracteristica) + '|' + str(etiqueta)]
                        probabilidad = st.norm.cdf(valorZ + margen) - st.norm.cdf(valorZ - margen)
                        resultados[indiceEtiqueta] *= probabilidad

        y_pred[i,0] = etiquetas[np.argmax(resultados)]
    return y_pred

In [241]:
pruebas = pd.read_csv("datos_validacion_temp.csv")
caracteristicas = pruebas.iloc[:,:-1]
etiquetas = pruebas.iloc[:,-1]

In [242]:
# Revisar métricas de datos de prueba
prediccionesArbol = modeloArbolDecision.predict(caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesArbol))
print(mts.f1_score(etiquetas, prediccionesArbol))
print(mts.precision_score(etiquetas, prediccionesArbol))
print(mts.recall_score(etiquetas, prediccionesArbol))

0.8321678321678322
0.8095238095238094
0.8225806451612904
0.796875


In [243]:
# Revisar métricas de datos de prueba
prediccionesSVM = modeloSVM.predict(caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesSVM))
print(mts.f1_score(etiquetas, prediccionesSVM))
print(mts.precision_score(etiquetas, prediccionesSVM))
print(mts.recall_score(etiquetas, prediccionesSVM))

0.8181818181818182
0.7936507936507936
0.8064516129032258
0.78125


In [244]:
# Revisar métricas de datos de prueba
prediccionesRegresion = predecirRegresionLogistica(caracteristicas, parametrosRegresionLogistica)
print(mts.accuracy_score(etiquetas, prediccionesRegresion))
print(mts.f1_score(etiquetas, prediccionesRegresion))
print(mts.precision_score(etiquetas, prediccionesRegresion))
print(mts.recall_score(etiquetas, prediccionesRegresion))

0.8391608391608392
0.8130081300813008
0.847457627118644
0.78125


In [246]:
# Revisar métricas de datos de prueba
prediccionesNaiveBayes = predecirNaiveBayes(probabilidadesNaiveBayes, caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesNaiveBayes))
print(mts.f1_score(etiquetas, prediccionesNaiveBayes))
print(mts.precision_score(etiquetas, prediccionesNaiveBayes))
print(mts.recall_score(etiquetas, prediccionesNaiveBayes))

0.7762237762237763
0.7460317460317459
0.7580645161290323
0.734375


## Función de predicción

In [253]:
# Predicción final combinando los modelos: Árbol de decisión, SVM
def prediccionFinal(X):
    prediccionesArbolDecision = modeloArbolDecision.predict(X)
    prediccionesSVM = modeloSVM.predict(X)
    prediccionesRegresionLogistica = predecirRegresionLogistica(X, parametrosRegresionLogistica)
    prediccionesNaiveBayes = predecirNaiveBayes(probabilidadesNaiveBayes, X).reshape(X.shape[0])
    prediccionTestArbolDecision = 0.8322
    prediccionTestSVM = 0.8182
    prediccionTestRegresionLogistica = 0.8130
    prediccionTestNaiveBayes = 0.7762
    totalPrediccion = prediccionTestArbolDecision + prediccionTestSVM + prediccionTestRegresionLogistica + prediccionTestNaiveBayes
    pesoArbolDecision =  prediccionTestArbolDecision / totalPrediccion
    pesoSVM = prediccionTestSVM / totalPrediccion
    pesoRegresionLogistica = prediccionTestRegresionLogistica / totalPrediccion
    pesoNaiveBayes = prediccionTestNaiveBayes / totalPrediccion
    prediccionFinal = pesoArbolDecision * prediccionesArbolDecision \
                    + pesoSVM * prediccionesSVM \
                    + pesoRegresionLogistica * prediccionesRegresionLogistica \
                    + pesoNaiveBayes * prediccionesNaiveBayes
    return np.round(prediccionFinal)

In [254]:
# Revisar métricas de datos de prueba
prediccionesFinales = prediccionFinal(caracteristicas)
print(prediccionesFinales)
print(mts.accuracy_score(etiquetas, prediccionesFinales))
print(mts.f1_score(etiquetas, prediccionesFinales))
print(mts.precision_score(etiquetas, prediccionesFinales))
print(mts.recall_score(etiquetas, prediccionesFinales))

(143,) (143,) (143,) (143,)
[0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
0.8251748251748252
0.8
0.819672131147541
0.78125


In [107]:
modeloNaiveBayes = np.load('test.npy', allow_pickle=True)
#diccTest = dict(enumerate(modeloNaiveBayes[0].flatten(), 1))[1]
#diccTest['0_0']
modeloNaiveBayes[0]['1']

0.4

## Predicción árbol de decisión

## Predicción Support Vector Machine (SVM)

## Predicción Naive Bayes

## Predicción regresión logística

## Validación